In [26]:
from corpus_reader import read_dataset
#from n_gram_naive_bayes import trained_naive_bayes, extract_features_and_classify

from word_extraction import remove_puntuations
from numpy.random import choice

from sklearn import svm

import sys
import numpy as np

import re

In [20]:
data = read_dataset()
n_docs = len(data)

np.random.seed(42)

train_amount = 0.7
train_ids = choice(n_docs, int(train_amount * n_docs), replace=False)
test_ids = [i for i in range(n_docs) if i not in train_ids]

training = [data[i] for i in train_ids]
test = [data[i] for i in test_ids]

print("%i training documents" % len(training))
print("%i test documents" % len(test))


499 training documents
215 test documents


In [ ]:
def str_pair(pair):
    typeInteraction = ""
    
    if pair.ddi == "true":
        typeInteraction = "," + pair.type
    
    return "(" + pair.e1.text + "," +  pair.e2.text + "," + pair.ddi + typeInteraction + ")"

In [21]:
def get_pairs_interaction(pair):
    return "(" + pair.e1.text + "," +  pair.e2.text + "," + pair.ddi + ")"


In [54]:
def get_interaction_text(pair, sentenceText):
    
    if len(pair.e1.char_offset) != 1:
        print("MAUVAIS CAS !!!")
        print (pair.e1.char_offset)
        print(sentenceText)
        print(pair.e1.text)
        print(pair.e2.text)
        return []
        sys.exit()
    
    
    #print (pair.e1.char_offset, pair.e2.char_offset)
    text_start = pair.e1.char_offset[0][1] +1
    text_end = pair.e2.char_offset[0][0]        #no +1 because slice in python exclude last end
    
    #print "TEXT START AT " + str(text_start) + " AND ENDS AT " + str(text_end) + "\n"
    
    text = sentenceText[text_start:text_end]
    text = re.split("\W+", text)
    text = [w for w in text if w != ""]
    #text = "--"
    #return "(" + pair.e1.text + "," + str(text) + "," +  pair.e2.text + "," + pair.ddi + ")"
    return text


In [55]:
nb=1
nb = len(training)

count_words = {}

verbose = False

for doc in training[0:nb]:
    for sentence in doc.sentences:
        if len(sentence.entities) >= 2:
            all_interaction_text = [get_interaction_text(p, sentence.text) for p in sentence.pairs]
            if verbose:
                print(sentence.text)
                print([e.text for e in sentence.entities])
                #print([get_pairs_interaction(p) for p in sentence.pairs])

                print(interaction_text)
                print("\n-------\n")
            
            for interaction_text in all_interaction_text:
                for w in interaction_text:
                    count_words[w] = count_words.get(w,0) + 1
            
            
            
print("Done")

MAUVAIS CAS !!!
[(81, 93), (112, 122)]
The beneficial effects on arterial thrombus formation from combined therapy with antiplatelet and anticoagulant medication must be weighed against an increased risk of inducing hemorrhage.
antiplatelet medication
anticoagulant medication
MAUVAIS CAS !!!
[(94, 98), (114, 122)]
Vaccines: Patients on corticosteroid therapy may exhibit a diminished response to toxoids and live or inactivated vaccines due to inhibition of antibody response.
live vaccines
inactivated vaccines
MAUVAIS CAS !!!
[(150, 154), (188, 197)]
In some patients, the administration of a non- steroidal antiinflammatory agent can reduce the diuretic, natriuretic, and antihypertensive effects of loop, potassium- sparing and thiazide diuretics.
loop diuretics
potassium- sparing diuretics
MAUVAIS CAS !!!
[(150, 154), (188, 197)]
In some patients, the administration of a non- steroidal antiinflammatory agent can reduce the diuretic, natriuretic, and antihypertensive effects of loop, potas

In [56]:
srt = sorted([(count_words[word], word) for word in count_words], reverse = True)

In [63]:
nb_feature_words = 20
print("Showing most "+ str(nb_feature_words) + " frequent words.")

for e in srt[:nb_feature_words]:
    print(e)

Showing most 20 frequent words.
(6619, 'and')
(6008, 'of')
(4383, 'the')
(3519, 'acid')
(3497, 'with')
(2929, 'drugs')
(2475, 'or')
(2366, 'in')
(2341, 'inhibitors')
(1794, 'as')
(1766, 'to')
(1507, 'may')
(1440, 'a')
(1277, 'oral')
(1196, 'phenytoin')
(1194, 'mg')
(1176, 'prolonged')
(1142, 'clofibrate')
(1132, 'g')
(1125, 'e')


In [66]:
feature_words = [w for (n,w) in srt[:nb_feature_words]]
feature_index = {}

for (i,feat) in enumerate(feature_words):
    feature_index[feat] = i

In [67]:
feature_index

{'a': 12,
 'acid': 3,
 'and': 0,
 'as': 9,
 'clofibrate': 17,
 'drugs': 5,
 'e': 19,
 'g': 18,
 'in': 7,
 'inhibitors': 8,
 'may': 11,
 'mg': 15,
 'of': 1,
 'or': 6,
 'oral': 13,
 'phenytoin': 14,
 'prolonged': 16,
 'the': 2,
 'to': 10,
 'with': 4}

In [130]:
#columns = feature_words (1 = present, 0 = absent)
#row = pair_n

def get_features_from_text(wordList):
    output = np.zeros(nb_feature_words)
    for w in wordList:
        if w in feature_index:
            output[feature_index[w]] = 1
    
    return output



def get_features_from_pair(pair, sentenceText):
    text_start = pair.e1.char_offset[0][1] +1
    text_end = pair.e2.char_offset[0][0]        #no +1 because slice in python exclude last end
    
    #print "TEXT START AT " + str(text_start) + " AND ENDS AT " + str(text_end) + "\n"
    
    text = sentenceText[text_start:text_end]
    text = re.split("\W+", text)
    text = " ".join([w for w in text if w != ""])
    
    return get_features_from_text(text)

def get_features_from_sentence(s):
    ncol = nb_feature_words
    nrow = len(s.pairs)
    
    matrix = np.zeros(shape=(nrow, ncol))
    
    for (i,p) in enumerate(s.pairs):
        matrix[i] = get_features_from_pair(p, s.text)
    
    return matrix

In [188]:

def get_label_from_pair(pair):
    label = "no_interaction"
    
    if pair.ddi == "true":
        label = pair.type
        
        if label == "None":
            print ("EWRWOREIWOPRWOREOWPEOPPWROEWPRWOREWOROWPREWOPRWOPRWOPRWEOPR")
    
    return label

def get_labels_from_sentence(s):
    ncol = 1
    nrow = len(s.pairs)
    
    matrix = [""]*nrow
    
    for (i,p) in enumerate(s.pairs):
        matrix[i] = get_label_from_pair(p)
    
    return np.array(matrix)

In [189]:
s = "And of course, acid with water is dangerous !"
s = s.lower().split(' ')
s

['and', 'of', 'course,', 'acid', 'with', 'water', 'is', 'dangerous', '!']

In [190]:
get_features_from_text(s)

array([ 1.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [191]:
def get_necessary_info(pair, sentenceText):
    if len(pair.e1.char_offset) != 1:
        #print("MAUVAIS CAS !!!")
        #print (pair.e1.char_offset)
        #print(sentenceText)
        #print(pair.e1.text)
        #print(pair.e2.text)
        #return []
        #sys.exit()
        ()
    
    #print (pair.e1.char_offset, pair.e2.char_offset)
    text_start = pair.e1.char_offset[0][1] +1
    text_end = pair.e2.char_offset[0][0]        #no +1 because slice in python exclude last end
    
    #print "TEXT START AT " + str(text_start) + " AND ENDS AT " + str(text_end) + "\n"
    
    text = sentenceText[text_start:text_end]
    text = re.split("\W+", text)
    text = " ".join([w for w in text if w != ""])
    #text = "--"
    #return "(" + pair.e1.text + "," + str(text) + "," +  pair.e2.text + "," + pair.ddi + ")"
    #return text
    
    typeInteraction = ""
    
    if pair.ddi == "true":
        typeInteraction = "," + pair.type
    
    return "(" + pair.e1.text + "," + text + "," +  pair.e2.text + "," + pair.ddi + typeInteraction + ")"


In [192]:
nb = 1
nb = len(training)
verbose = False

ncol = nb_feature_words
nrow = 1

matrixFeature = np.zeros(shape=(nrow, ncol))

labels = np.array([])



for doc in training[0:nb]:
    for sentence in doc.sentences:
        if len(sentence.entities) >= 2:
            #all_interaction_text = [get_interaction_text(p, sentence.text) for p in sentence.pairs]
            matrixFeature = np.vstack((matrixFeature, get_features_from_sentence(sentence)))
            labels = np.append(labels, get_labels_from_sentence(sentence))
            if verbose:
                print(sentence.text)
                print([e.text for e in sentence.entities])
                #print([get_necessary_info(p, sentence.text) for p in sentence.pairs])
            
                
                #print(all_interaction_text)
                print("\n-------\n")
            

matrixFeature = matrixFeature[1:,:] #remove first row
labels = labels.T
print(matrixFeature.shape)
#print(matrixFeature)
print(np.unique(labels))
print("Done")

(20869L, 20L)
[None 'advise' 'effect' 'int' 'mechanism' 'no_interaction']
Done


In [186]:
sl = 2
step = 60
labels[sl*step:(sl+1)*step]

t = {}
for e in labels:
    t[e] = t.get(e,0) + 1

In [187]:
t

{None: 1,
 'advise': 599,
 'effect': 1204,
 'int': 138,
 'mechanism': 990,
 'no_interaction': 17937}